In [3]:
urls = ["https://www.lighting.philips.com/content/b2b-philips-lighting/vn/vi/products/highlighted-products/ledstrips",
       "https://www.lighting.philips.com/content/b2b-philips-lighting/ph/en/support/cases/airports/schiphol-airport",
        "https://www.lighting.philips.com/content/b2b-philips-lighting/ph/en/products/highlighted-products/ledstrips",
        "https://www.lighting.philips.com/content/b2b-philips-lighting/vn/vi/products/highlighted-products/maxos-fusion",
        "https://www.lighting.philips.com/content/b2b-philips-lighting/ph/en/application-areas/public-spaces/recreational-sports/outdoor-sports"
]

In [11]:
import pandas as pd

In [12]:
df = pd.read_csv("C:/Users/Usuario/Documents/marketo_python/urls_revisar_error404.csv")
df.head(5)

,urls
0,https://www.lighting.philips.com/content/b2b-p...
1,https://www.lighting.philips.com/content/b2b-p...
2,https://www.lighting.philips.com/content/b2b-p...
3,https://www.lighting.philips.com/content/b2b-p...
4,https://www.lighting.philips.com/content/b2b-p...


In [16]:
urls = df["urls"].iloc[200:477].tolist()

In [6]:
## Código que abre y cierra el navegador fantasma

import asyncio
import nest_asyncio
import csv
from playwright.async_api import async_playwright

nest_asyncio.apply()

async def detectar_error_404(page, url):
    print(f"\n🌐 Procesando URL: {url}")
    error_404_detectado = False

    def manejar_respuesta(response):
        nonlocal error_404_detectado
        if response.status == 404:
            print(f"❌ Recurso 404 detectado: {response.url}")
            error_404_detectado = True

    page.on("response", manejar_respuesta)

    try:
        await page.goto(url, timeout=20000)
        await page.wait_for_timeout(3000)
    except Exception as e:
        print(f"⚠️ Error al cargar la página: {e}")
        return True  # Lo consideramos como error también

    return error_404_detectado

async def main():
    # Lista de URLs

    resultados = []

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=False)
        context = await browser.new_context()

        for url in urls:
            page = await context.new_page()
            resultado = await detectar_error_404(page, url)
            resultados.append({"url": url, "error_404": resultado})
            await page.close()

        await browser.close()

    # Guardar resultados en CSV
    with open("resultados_errores_404.csv", mode="w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=["url", "error_404"])
        writer.writeheader()
        writer.writerows(resultados)

    print("\n✅ Resultados guardados en 'resultados_errores_404.csv'")
 

if __name__ == "__main__":
    asyncio.run(main())



🌐 Procesando URL: https://www.lighting.philips.com/content/b2b-philips-lighting/vn/vi/products/highlighted-products/ledstrips
❌ Recurso 404 detectado: https://www.lighting.philips.com/content/b2b-philips-lighting/vn/vi/products/highlighted-products/ledstrips

🌐 Procesando URL: https://www.lighting.philips.com/content/b2b-philips-lighting/ph/en/support/cases/airports/schiphol-airport

🌐 Procesando URL: https://www.lighting.philips.com/content/b2b-philips-lighting/ph/en/products/highlighted-products/ledstrips
❌ Recurso 404 detectado: https://www.lighting.philips.com/content/b2b-philips-lighting/ph/en/products/highlighted-products/ledstrips

🌐 Procesando URL: https://www.lighting.philips.com/content/b2b-philips-lighting/vn/vi/products/highlighted-products/maxos-fusion
❌ Recurso 404 detectado: https://www.lighting.philips.com/content/b2b-philips-lighting/vn/vi/products/highlighted-products/maxos-fusion

🌐 Procesando URL: https://www.lighting.philips.com/content/b2b-philips-lighting/ph/en/

In [17]:
### Código definitivo 

import asyncio
import nest_asyncio
import csv
from playwright.async_api import async_playwright

nest_asyncio.apply()

# Lista de URLs a revisar (actualizar lista en la parte superior del cuaderno)


async def revisar_error_404(page, url):
    print(f"\n🌐 Revisando: {url}")
    error_404_detectado = False

    async def manejar_respuesta(response):
        nonlocal error_404_detectado
        if response.status == 404:
            print(f"❌ Recurso 404 detectado: {response.url}")
            error_404_detectado = True

    page.on("response", manejar_respuesta)

    try:
        await page.goto(url, timeout=20000)
        await page.wait_for_timeout(3000)
    except Exception as e:
        print(f"⚠️ Error al cargar la página: {e}")
        return True  # Consideramos que hubo un error

    return error_404_detectado

async def main():
    resultados = []

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=False)
        context = await browser.new_context()

        # Pestaña en blanco que permanece abierta
        pestana_principal = await context.new_page()
        paginas_abiertas = [pestana_principal]

        for url in urls:
            nueva_pestana = await context.new_page()
            paginas_abiertas.append(nueva_pestana)

            if len(paginas_abiertas) > 2:
                anterior = paginas_abiertas.pop(1)
                await anterior.close()

            resultado = await revisar_error_404(nueva_pestana, url)
            resultados.append({"url": url, "error_404_detectado": resultado})

        print("\n📝 Guardando resultados...")

        with open("resultados_errores404_2.csv", mode="w", newline="", encoding="utf-8") as f:
            writer = csv.DictWriter(f, fieldnames=["url", "error_404_detectado"])
            writer.writeheader()
            writer.writerows(resultados)

        print("✅ Resultados guardados en 'resultados_errores404_2.csv'")

        # El navegador NO se cierra aquí a propósito

if __name__ == "__main__":
    asyncio.run(main())



🌐 Revisando: https://www.lighting.philips.com/content/b2b-philips-lighting/aa/en/application-areas/specialist-applications/horticulture/hortiblog/vegetables-and-fruits/improves-shelf-life-and-vitamins-in-your-lettuce

🌐 Revisando: https://www.lighting.philips.com/content/b2b-philips-lighting/aa/en/application-areas/specialist-applications/horticulture/hortiblog/vegetables-and-fruits/why-avoid-white-light-in-basil-cultivation

🌐 Revisando: https://www.lighting.philips.com/content/b2b-philips-lighting/aa/en/application-areas/specialist-applications/horticulture/hortiblog/light-and-growth/myth-surrounding-white-light

🌐 Revisando: https://www.lighting.philips.com/content/b2b-philips-lighting/aa/en/application-areas/specialist-applications/horticulture/hortiblog/light-and-growth/myth-surrounding-white-light-part-3

🌐 Revisando: https://www.lighting.philips.com/content/b2b-philips-lighting/aa/en/application-areas/specialist-applications/horticulture/hortiblog/light-and-growth/shift-from-op